<a href="https://colab.research.google.com/github/IlyaKuprik/skolkovo-hack/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Process

## загрузка


In [ ]:
{"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [199]:
data_candidates_work_places = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_work_places.csv', sep=';',
                                names = ['CandidateId', 'Position','FromYear', 'FromMonth', 'ToYear', 'ToMonth'])

In [200]:
data_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/data_jobs.csv', sep=';',
                                names = ['JobId', 'Status','Name', 'Region', 'Description'])

In [201]:
data_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates.csv', sep=';',
                                names = ['CandidateId', 'Position','Sex', 'Citizenship', 'Age',
                                'Salary', 'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Shedule',                                           
                                 'CandidateRegion', 'DateCreated', 'JobId', 'CandidateStatusId', 'Status'])

In [202]:
data_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_education.csv', sep=';',
                                names = ['CandidateId', 'University', 'Faculty', 'GraduateYear'])

In [203]:
data_candidates.duplicated(['Position', 'Citizenship', 'DriverLicense', 'Employment', 'Shedule', 'CandidateRegion']).sum()

128345

## Обработка таблицы

In [204]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
data_jobs.Description = data_jobs.Description.apply(cleanhtml)

In [205]:
cols_val = {str(i): 'last' for i in data_candidates.columns}

In [206]:
grouped_with_last_status = data_candidates.sort_values(by = 'DateCreated').loc[:,data_candidates.columns]\
.groupby(by = ['CandidateId','JobId'], axis = 0, sort = False, as_index = False).agg( cols_val)

### Штрафы

In [207]:
target_map = {"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [208]:
targets = grouped_with_last_status['Status'].apply(lambda x: target_map[x] if x in target_map else np.NaN) ## Заполнили NA
grouped_with_ids_target = grouped_with_last_status.loc[:, ['CandidateId', 'JobId',]]
grouped_with_ids_target['target'] = targets

In [209]:
#Инициализация штрафов

number_fields = ['Age', 'Salary']
string_fields = ['Position', 'Citizenship', 'Langs',\
                  'Skills', 'Employment', 'Shedule', 'CandidateRegion']
fines_vector = [1/9]*9 

In [210]:

mask_df = grouped_with_last_status.copy()
mask_df[number_fields] = grouped_with_last_status[number_fields].apply(lambda x: x == 0)
mask_df[string_fields] = grouped_with_last_status[string_fields].apply(lambda x: x.isna())

total_fine = mask_df[number_fields + string_fields].apply(lambda x: 1 - (np.array(x) * np.array(fines_vector)).sum(), axis = 1)
total_target = targets*total_fine
grouped_with_ids_target['target'] = total_target

## Driver license

In [211]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text

data_candidates.DriverLicense = data_candidates.DriverLicense.apply(del_sep)
data_candidates.DriverLicense.isna().sum()

93730

In [212]:

# for i, row in data_candidates.iterrows():
#     if type(row.DriverLicense) != type('') and type(row.Position) == type(''):
#         if np.isnan(row.DriverLicense):
#             Flag = False
#             if 'водитель' in row.Position.lower():
#                 if not Flag:
#                     data_candidates.iloc[i, 7] = 'B'
#                     Flag = True
#             if 'грузового' in row.Position.lower() or 'грузовой' in row.Position.lower():
#                 if not Flag:
#                     data_candidates.iloc[i, 7] = 'C'
#                     Flag = True
#                 else:
#                     data_candidates.iloc[i, 7] += 'C'
#             if Flag == False:
#                 data_candidates.iloc[i, 7] = 'N'
# data_candidates.DriverLicense.isna().sum()

## citizenship

In [ ]:
data_candidates.Citizenship

In [ ]:
data_candidates.Citizenship.apply()

In [ ]:
data_candidates.Citizenship.apply(lambda x: 'Россия' if 'рос' in x.lower() or 'рф' in x.lower() or 'rus' in x.lower()\
                                  else 'Киргизия' if x.lower().startswith('к')\
                                  else 'Армения' if x.lower().startswith('а')\
                                  else 'Узбекистан' if x.lower().startswith('у')
                                  else 'Таджикистан')

In [ ]:
for i, row in data_candidates.iterrows():
    if type(row.Citizenship) == type(''):
        if 'рос' in row.Citizenship.lower() or 'рф' in row.Citizenship.lower() or 'rus' in row.Citizenship.lower():
            data_candidates.iloc[i, 3] = 'Россия'
        if row.Citizenship.lower().startswith('к'):
            data_candidates.iloc[i, 3] = 'Киргизия'
        if row.Citizenship.lower().startswith('а'):
            data_candidates.iloc[i, 3] = 'Армения'
        if row.Citizenship.lower().startswith('у'):
            data_candidates.iloc[i, 3] = 'Узбекистан'
        if row.Citizenship.lower().startswith('т'):
            data_candidates.iloc[i, 3] = 'Таджикистан'

## Skills

In [214]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text


data_candidates.Skills = data_candidates.Skills.apply(del_stick)

### Обработка текста

In [215]:
grouped_table_candidates_features = grouped_with_last_status.drop(columns = ['Status', 'CandidateStatusId'])

In [ ]:
grouped_table_candidates_features

In [ ]:
candidates_work_places_concat = data_candidates_work_places.drop_duplicates(['CandidateId', 'Position'])
candidates_work_places_concat.Position = candidates_work_places_concat.Position.add(' ')
candidates_work_places_concat = candidates_work_places_concat.groupby('CandidateId', as_index=False).agg(Position_last_work = ('Position' , 'sum'))
candidates_work_places_concat

In [217]:
new_df = grouped_table_candidates_features\
    .merge(candidates_work_places_concat, on='CandidateId', how='left')\
    .merge(data_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')

In [218]:
new_df = new_df.drop_duplicates(['Position', 'Citizenship', 'DriverLicense',\
                        'Employment', 'Shedule', 'CandidateRegion',\
                        'Position_last_work', 'University', 'Faculty' ])

In [ ]:
new_df

In [219]:

new_df.index = [new_df.CandidateId, new_df.JobId]
df = new_df.drop(columns=['JobId', 'CandidateId', 'Salary', 'Subway',
                          'Age', 'GraduateYear', 'DateCreated'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()
 
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = data_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()
 
job_user_tokens2 = df
job_user_tokens_with_id = job_user_tokens2\
    .merge(job_user_tokens1, on='JobId')
 

In [220]:
merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
merged_descriptions_targets_ids = merged_descriptions_targets_ids[~(merged_descriptions_targets_ids.target.isna())]

In [ ]:
# merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
# merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
# merged_descriptions_targets_ids.isna().sum()
# merged_descriptions_targets_ids = merged_descriptions_targets_ids.fillna(1/2)

In [221]:
merged_descriptions_targets_ids.drop(columns = ['CandidateId', 'JobId'], inplace = True)

In [224]:
merged_descriptions_targets_ids.drop_duplicates(['Candidate_descr', 'Jobs_descr'])

,Candidate_descr,Jobs_descr,target
0,Водитель. мужчина. Гражданство. NaN. NaN. NaN....,Водитель. Санкт-Петербург. Требования к опыту...,0.277778
1,Водитель. неопределен. NaN. NaN. NaN. NaN. Ful...,Водитель. Санкт-Петербург. Требования к опыту...,0.316667
2,Водитель. мужчина. Россия. NaN. NaN. NaN. Full...,Водитель. Санкт-Петербург. Требования к опыту...,0.277778
3,Водитель. мужчина. Россия. NaN. NaN. NaN. Full...,Водитель. Санкт-Петербург. Требования к опыту...,0.277778
4,Водитель. мужчина. Россия. NaN. NaN. NaN. Full...,Водитель. Санкт-Петербург. Требования к опыту...,0.277778
...,...,...,...
24171,водитель курьер на авто . мужчина. 89262820275...,Водитель на авто компании . Москва. NaN. откры...,0.444444
24175,Водитель погрузчика. мужчина. Россия. русский:...,Водитель погрузчика / штабелера. Москва. Водит...,0.000000
24176,Кладовщик-комплектовщик. мужчина. Россия. русс...,Водитель погрузчика / штабелера. Москва. Водит...,0.700000
24177,"Водитель погрузчика, Штабелера. мужчина. Росси...",Водитель погрузчика / штабелера. Москва. Водит...,0.622222


In [225]:
merged_descriptions_targets_ids = merged_descriptions_targets_ids.drop_duplicates(['Candidate_descr', 'Jobs_descr'])

In [226]:
merged_descriptions_targets_ids.to_csv('preprocess_data.csv')

# Обработка теста


In [111]:
test_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates.csv', sep = ';')

In [112]:
test_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_jobs.csv', sep = ';',\
                        names = ['JobId', 'Status','Name', 'Region', 'Description', 'x', 'y','z'])
test_jobs.drop(columns = ['x', 'y','z'], inplace = True)

In [113]:
test_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates_education.csv', sep=';')

In [114]:
test_candidates_workplaces = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates_workplaces.csv', sep = ';')

## Обработка Жопы

In [115]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
test_jobs.Description = test_jobs.Description.apply(cleanhtml)

In [116]:
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = test_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()

## Обработка бывшего места работы

In [ ]:
candidates_work_places_concat = test_candidates_workplaces.drop_duplicates(['CandidateId', 'Position'])
candidates_work_places_concat.Position = candidates_work_places_concat.Position.add(' ')
candidates_work_places_concat = candidates_work_places_concat.groupby('CandidateId', as_index=False).agg({'Position' : 'sum'})
candidates_work_places_concat

## Обработка кандидата

In [118]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text

test_candidates.DriverLicense = test_candidates.DriverLicense.apply(del_sep)
test_candidates.DriverLicense.isna().sum()

90

In [119]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text


test_candidates.Skills = test_candidates.Skills.apply(del_stick)

In [120]:
new_df = test_candidates\
    .merge(candidates_work_places_concat, on='CandidateId', how='left')\
    .merge(test_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
new_df.index = new_df.CandidateId
df = new_df.drop(columns=['CandidateId', 'Salary', 'Subway',
                          'Age', 'GraduateYear'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()

job_user_tokens2 = df

In [121]:
job_user_tokens2 = job_user_tokens2.rename(columns = {0: 'Candidate_descr'})
job_user_tokens1 = job_user_tokens1.rename(columns = {0: 'Jobs_descr'})
job_user_tokens2.drop(columns = ['CandidateId'], inplace = True)
job_user_tokens1.drop(columns = ['JobId'], inplace = True)



In [125]:
job_user_tokens1

,Jobs_descr
0,Водитель автомобиля большегруза с прицепом . С...
1,водитель на своем авто . Москва. Начался сезон...
2,Водитель на своем авто. Москва. Стаж вождения ...
3,Водитель. Санкт-Петербург. Обязанности:Качеств...
4,водитель на личном авто. Москва. ищем курьера ...
5,Водитель-экспедитор 1/ Водитель. Москва. Обяза...
6,водитель курьер на авто компании. Москва. води...
7,Менеджер по продажам. Санкт-Петербург. Обязанн...
8,Водитель-экспедитор. Москва. ЗАДАЧИ: Доставка...
9,"Водитель. Москва. Наш клиент, крупный холдинг ..."


In [124]:
job_user_tokens2

,Candidate_descr
0,специалист отдела парковочного комплекса. мужч...
1,Оператор. мужчина. Россия. русский:родной||нем...
2,"Ученик автослесаря , автомеханика .. мужчина. ..."
3,Агент по обработке авиаперевозок. мужчина. Рос...
4,Техник-механик. мужчина. Россия. русский:родно...
...,...
195,"Менеджер по продажам, ведущий менеджер. женщин..."
196,"Менеджер по продажам, закупкам. мужчина. Росси..."
197,Менеджер по продажам. мужчина. Россия. русски...
198,Менеджер по продажам. мужчина. Россия. русски...


In [126]:
test_data_merged = job_user_tokens2.merge(job_user_tokens1, how = 'cross')

In [127]:
test_data_merged.duplicated(['Candidate_descr', 'Jobs_descr']).sum()

10

In [128]:
test_data_merged.to_csv('test_data_merged.csv')